In [1]:
# needed libraries 

import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import string
import os
from translate import Translator

## Normalizer

In [2]:
# removing puctution
def remove_punctuation(text):
    punctuation = string.punctuation
    text_without_punctuation = ''.join(char for char in text if char not in punctuation)
    return text_without_punctuation


# removing duplicated letters such (example: 'ijaaaaaa' -> 'ija'  'nheeeeebeeek' -> 'nhebek')
def remove_similar_letters(word):
    if not word:
        return word

    i = 0
    result = word[0]

    while i < len(word) - 1:
        if word[i] != word[i + 1]:
            result += word[i + 1]
        i += 1

    return result

## Converting tokens to every possible output

In [3]:
# stemming words

def stemmer(word):
    first_2 =  ['مي', 'مت','ال']
    first_1 = ['ا', 'ي', 'ت',  'ن','م']
    last_1 = ['و', 'ك','ا','ي']
    last_2 = ['نا', 'ني', 'او', 'لو', 'لي', 'لك', 'تو', 'اك''كم', 'يو','هم','وه','يه']
    last_3 = ['وهم','وكم','لنا', 'لكم', 'لهم','يهم','فال']

    removed = False
    # case of negative word (example: 'مايجيكمش')
    if word.startswith('ما') and word.startswith('ش') and len(word)>4 :
        word = word[2:len(word)-1]
    for prefix in first_2:
        if word.startswith(prefix) and len(word)>3:
            word = word[2:]
            removed = True
            break
    if not(removed):
        for prefix in first_1:
            if word.startswith(prefix) and len(word)>4:
                word = word[1:]
                break
    removed = False
    for prefix in last_3:
        if word.endswith(prefix) and len(word)>4:
            word = word[0:len(word)-3]
            removed = True
            break
    if not(removed):
        for prefix in last_2:
            if word.endswith(prefix) and len(word)>3:
                word = word[0:len(word)-2]
                removed = True
                break
    if not(removed):
        for prefix in last_1:
            if word.endswith(prefix) and len(word)>2:
                word = word[0:len(word)-1]
                break
    return word

# function that removes dublicates from array with keeping their initial order (we will use it in next function)
def remove_duplicates_and_keep_order(arr):
    unique_elements = []
    for item in arr:
        if item not in unique_elements:
            unique_elements.append(item)
    return unique_elements


#convert each letter in word with matching letters and return all combinations
def get_combinations(word):
    if (word.startswith('ou') and len(word)>2):
        word = 'ا' + word[2:]
    if (word.startswith('i') or word.startswith('o')):
        word = 'ا' + word[1:]
    if word.endswith('a'):
        word =  word[0:len(word)-1] + 'ا'
    if word.endswith('o'):
        word =  word[0:len(word)-1] + 'و'
    word = word.replace('ch','ش')
    word = word.replace('gh','غ')
    word = word.replace('ou','و')
    word = word.replace('aa','ع')
    word = remove_similar_letters(word)
    combinations = ['']
    latin = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z','2','3','5','7','9','8']
    # most common
    arabic = ['ا', 'ب', 'س', 'د', 'ا', 'ف', 'ق', 'ه', 'ي', 'ج', 'ك', 'ل', 'م', 'ن', '', 'ب', 'ق', 'ر', 'س', 'ت', 'و', 'ف', 'و', 'و', 'ي', 'ز','أ','ع','خ','ح','ق','غ']
    # less common
    arabic2 = ['ع', 'ب', 'ك', 'د', '', 'ف', 'ق', 'ه', 'ي', 'ج', 'ق', 'ل', 'م', 'ن', '', 'ب', 'ك', 'ر', 'ص', 'ط', 'و', 'ف', 'و', 'و', 'ي', 'ز','أ','ع','خ','ح','ق','غ']
    # less common
    arabic3 = ['', 'ب', 'س', 'د', 'ي', 'ف', 'ج', 'ح', 'ي', 'ج', 'ق', 'ل', 'م', 'ن', '', 'ب', 'ك', 'ر', 'س', 'ت', '', 'ف', 'و', '', 'ي', 'ز','أ','ع','خ','ح','ق','غ']
    for i in word:
        temp_combinations = []
        if not(i.lower() in latin):
            for combination in combinations:
                temp_combinations.append(combination + i)
            combinations = remove_duplicates_and_keep_order(temp_combinations)
            continue
        
        letter_index = latin.index(i.lower())
        for combination in combinations:
            temp_combinations.append(remove_similar_letters(combination + arabic[letter_index]))
            temp_combinations.append(remove_similar_letters(combination + arabic2[letter_index]))
            temp_combinations.append(remove_similar_letters(combination + arabic3[letter_index]))
        combinations = remove_duplicates_and_keep_order(temp_combinations)
        skip_next = False
        for combination in combinations:
            if skip_next:
                skip_next = False
                continue
            # other possible combinations
            if combination.find('ته')>-1:
                combinations.insert(combinations.index(combination),combination.replace('ته','ث'))
                skip_next = True
            if combination.find('كه')>-1:
                combinations.insert(combinations.index(combination)+1,combination.replace('كه','خ'))
            if combination.find('ده')>-1:
                combinations.insert(combinations.index(combination)+1,combination.replace('ده','ض'))
            if combination.find('ده')>-1:
                combinations.insert(combinations.index(combination)+1,combination.replace('ده','ذ'))
    return remove_duplicates_and_keep_order(combinations)



## Convert sentence to 

In [19]:
#check if the word is all in arabic letters
def is_arabic(word):
    arabic_letters = []
    start = ord('\u0600')  # Start of Arabic Unicode range
    end = ord('\u06FF')  # End of Arabic Unicode range
    for codepoint in range(start, end + 1):
        character = chr(codepoint)
        arabic_letters.append(character)
    for i in word:
        if not(i in arabic_letters):
            return False
    return True


# check if word exists in the vocabulary file

file_path = r'C:\Users\Admin\Desktop\dictionary.txt'
def exist(combinations):
    existing=[]
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            for combination in combinations:
                if combination == line.strip('\n'):
                    existing.insert(combinations.index(combination),combination)
                if stemmer(combination) in line.strip('\n'):
                    existing.append(combination)
        existing = remove_duplicates_and_keep_order(existing)
    return existing

# translation function
def translate_text(text, from_lang ,target_language):
    translator = Translator(from_lang=from_lang , to_lang=target_language )
    translation = translator.translate(text)
    return translation.split(',')[0]


# convert to arab
def convert_to_arab(post):
    new_post=[]
    french_special_letters = ['é', 'è', 'à', 'ù', 'â', 'ê', 'î', 'ô', 'û', 'ç']
    post = post.replace(' el ',' ال')
    post = post.replace(' l ',' ل')
    post = post.replace(' f ',' في ')
    post = remove_punctuation(post)
    for word in post.split(' '):
        word = word.lower()
        skip = False
        for letter in french_special_letters:
            # If the word is oviously french
            if letter in word.lower():
                new_post.append(translate_text(word,'FR','AR'))
                skip = True
                break
        if skip:
            continue
        if word.isdigit():
            new_post.append(word)
        elif is_arabic(word):
            new_post.append(word)
        elif ('v'in word or 'x' in word or 'p' in word):
            if is_arabic(translate_text(word,'FR','AR').split(' ')[0]):
                new_post.append(translate_text(word,'FR','AR'))
            elif is_arabic(translate_text(word,'EN','AR').split(' ')[0]):
                new_post.append(translate_text(word,'EN','AR'))
            else:
                combinations = get_combinations(word)
                arabic_convertion = exist(combinations)
                if len(arabic_convertion) != 0:
                    new_post.append(arabic_convertion[0])
                else:
                    new_post.append(combinations[0])
                
        else:
            combinations = get_combinations(word)
            arabic_convertion = exist(combinations)
            if len(arabic_convertion) != 0:
                new_post.append(arabic_convertion[0])
            else:
                if is_arabic(translate_text(word,'FR','AR').split(' ')[0]):
                    new_post.append(translate_text(word,'FR','AR'))
                elif is_arabic(translate_text(word,'EN','AR').split(' ')[0]):
                    new_post.append(translate_text(word,'EN','AR'))
                else:
                    new_post.append(combinations[0])
    post = ''
    for word in new_post:
        post = post+' '+ remove_punctuation(word)
    return post[1:]
            


In [5]:
convert_to_arab('nheeeeb internship nshalah hata maa 3atar')

'نحب التدريب الداخلي نسهالاه حتا معا عطار'

In [6]:
convert_to_arab('Aandy chhar nlama7 ama zayed lbhym bhym')

'عندي شهر نلمح اما زيد لبهيم بهيم'

In [12]:
convert_to_arab('ena ingénieur f paris wenti ?')

'انا مهندس في باريس ونتي '

# Interface

In [20]:
import pygame
import arabic_reshaper
from bidi.algorithm import get_display

pygame.init()

# Replace this line with the path to your Arabic font file
font_path = r"C:\Users\Admin\Downloads\Amoriah.ttf"
font = pygame.font.Font(font_path, 36)

WINDOW_WIDTH, WINDOW_HEIGHT = 1000, 600
WHITE = (255, 255, 255)
LIGHT_GRAY = (200, 200, 200)
BLACK = (0, 0, 0)
BORDER_COLOR = (100, 100, 100)
BORDER_WIDTH = 2
PADDING = 25

MIN_LABEL_WIDTH = 400
MAX_LABEL_WIDTH = 800

screen = pygame.display.set_mode((WINDOW_WIDTH, WINDOW_HEIGHT))
pygame.display.set_caption("Tunisian Dialect Converter")

input_text = ""
result_text = ""

# Replace this line with the appropriate import statement for the convert_to_arab function
# from module import convert_to_arab

def get_label_width(text):
    return min(max(MIN_LABEL_WIDTH, font.size(text)[0] + PADDING * 2), MAX_LABEL_WIDTH)

def draw_button(text, x, y, width, height, color):
    pygame.draw.rect(screen, color, (x, y, width, height))
    text_surface = font.render(text, True, BLACK)
    text_x = x + (width - text_surface.get_width()) // 2
    text_y = y + (height - text_surface.get_height()) // 2
    screen.blit(text_surface, (text_x, text_y))


running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_RETURN:
                result_text = convert_to_arab(input_text)
            elif event.key == pygame.K_BACKSPACE:
                input_text = input_text[:-1]
                result_text = ''
            else:
                input_text += event.unicode
        elif event.type == pygame.MOUSEBUTTONDOWN:
            # Check if the mouse click is on the "Generate" button
            mouse_x, mouse_y = pygame.mouse.get_pos()
            button_width = 200
            button_height = 50
            button_x = (WINDOW_WIDTH - button_width) // 2
            button_y = (WINDOW_HEIGHT - button_height) - PADDING * 2
            if button_x <= mouse_x <= button_x + button_width and button_y <= mouse_y <= button_y + button_height:
                convert_and_display_result()


    screen.fill(WHITE)

    # Calculate input label width based on the length of the text
    input_label_width = get_label_width(input_text)
    input_label_height = font.size(input_text)[1] + PADDING * 2

    # Center the input label horizontally and vertically
    input_label_x = (WINDOW_WIDTH - input_label_width) // 2
    input_label_y = (WINDOW_HEIGHT - input_label_height * 2 - PADDING * 2) // 2

    # Render the "Input" label
    input_label_text = font.render("Input:", True, BLACK)
    input_label_text_x = (WINDOW_WIDTH - input_label_text.get_width()) // 2
    input_label_text_y = input_label_y - PADDING/2  - input_label_text.get_height()
    screen.blit(input_label_text, (input_label_text_x, input_label_text_y))

    # Render the input text and its border
    input_surface = font.render(input_text, True, BLACK)
    input_rect = pygame.Rect(input_label_x, input_label_y, input_label_width, input_label_height)
    input_border_rect = input_rect.inflate(BORDER_WIDTH * 2, BORDER_WIDTH * 2)
    pygame.draw.rect(screen, BORDER_COLOR, input_border_rect, BORDER_WIDTH)
    pygame.draw.rect(screen, LIGHT_GRAY, input_border_rect.inflate(-BORDER_WIDTH, -BORDER_WIDTH))
    screen.blit(input_surface, (input_label_x + PADDING + BORDER_WIDTH, input_label_y + PADDING + BORDER_WIDTH))

    # Reshape the result text to display it correctly from right to left
    reshaped_result_text = arabic_reshaper.reshape(result_text)
    bidi_result_text = get_display(reshaped_result_text)

    # Calculate result label width based on the length of the text
    result_label_width = get_label_width(bidi_result_text)
    result_label_height = font.size(bidi_result_text)[1] + PADDING * 2

    # Center the result label horizontally and vertically
    result_label_x = (WINDOW_WIDTH - result_label_width) // 2
    result_label_y = input_label_y + input_label_height + PADDING * 3
    
    # Render the "Output" label
    output_label_text = font.render("Output:", True, BLACK)
    output_label_text_x = (WINDOW_WIDTH - output_label_text.get_width()) // 2
    output_label_text_y = result_label_y - PADDING/2  - output_label_text.get_height()
    screen.blit(output_label_text, (output_label_text_x, output_label_text_y))

    # Render the result text and its border
    result_surface = font.render(bidi_result_text, True, BLACK)
    result_rect = pygame.Rect(result_label_x, result_label_y, result_label_width, result_label_height)
    result_border_rect = result_rect.inflate(BORDER_WIDTH * 2, BORDER_WIDTH * 2)
    pygame.draw.rect(screen, BORDER_COLOR, result_border_rect, BORDER_WIDTH)
    pygame.draw.rect(screen, LIGHT_GRAY, result_border_rect.inflate(-BORDER_WIDTH, -BORDER_WIDTH))
    screen.blit(result_surface, (result_label_x + PADDING + BORDER_WIDTH, result_label_y + PADDING + BORDER_WIDTH))

    # Draw the "Generate" button
    button_width = 200
    button_height = 50
    button_x = (WINDOW_WIDTH - button_width) // 2
    button_y = (WINDOW_HEIGHT - button_height) - PADDING * 2
    draw_button("Generate", button_x, button_y, button_width, button_height, LIGHT_GRAY)

    pygame.display.flip()

pygame.quit()
